## Work
1. 試比較有 BN 在 Batch_size = 2, 16, 32, 128, 256 下的差異
2. 請嘗試將 BN 放在 Activation 之前，並比較訓練結果
3. 請於 BN 放在 Input Layer 後，並比較結果

In [ ]:
import os
import keras
import itertools
# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
train, test = keras.datasets.cifar10.load_data()

In [ ]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [ ]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [ ]:

def build_mlp():
    """Code Here
    建立你的神經網路
    """
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units = n_units, 
                                   activation = "relu", 
                                   name = "hidden_layer" + str(i + 1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units = n_units, 
                                   activation = "relu", 
                                   name = "hidden_layer" + str(i + 1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units = output_units, activation = "softmax", name = "output")(x)
    
    model = keras.models.Model(inputs = [input_layer], outputs = [out])
    
    return model

In [ ]:
"""Code Here
設定超參數
"""
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = [2, 16, 32, 128, 256]
MOMENTUM = 0.95

In [ ]:
results = {}
"""Code Here
撰寫你的訓練流程並將結果用 dictionary 紀錄
"""
for bs in BATCH_SIZE:
    model = build_mlp(input_shape = x_train.shape[1:])
    model.summary()
    optimiz = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov = True, momentum = MOMENTUM)
    
    odel.compile(loss = "categorical_crossentropy", metrics = ["accuracy"], optimizer = optimiz)
    model.fit(x_train, y_train, 
          epochs = EPOCHS, 
          batch_size = bs, 
          validation_data = (x_test, y_test), 
          shuffle = True)

    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    results[str(bs)] = {
        'train_loss' : train_loss,
        'valid_loss':valid_loss,
        'train_acc':train_acc,
        'valid_acc':valid_acc
    }

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
"""Code Here
將結果繪出
"""
color = ['r','g','b']
plt.figure(figsize = (8, 6))

for i,key in enumerate(results.keys()):
    plt.plot(range(len(results[key]['train_loss'])), results[key]['train_loss'], label = "train_loss" + key, color = color[i])
    plt.plot(range(len(results[key]['valid_loss'])), results[key]['valid_loss'], label = "val_loss" + key, color = color[i])
plt.legend()
plt.title("Loss")

plt.show()

for i,key in enumerate(results.keys()):
    plt.plot(range(len(results[key]['train_acc'])), results[key]['train_acc'], label = "train_acc" + key, color = color[i])
    plt.plot(range(len(results[key]['valid_acc'])), results[key]['valid_acc'], label = "val_acc" + key, color = color[i])
plt.legend()
plt.title("ACUURACY")

plt.show()